# DL model embedding (KERAS)


In [ ]:
# !pip install keras-tuner

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,GRU,BatchNormalization,Bidirectional,Dropout
from tensorflow.keras import Input as Input_layer
from tensorflow.keras.layers import Flatten
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Embedding
from tensorflow.keras.layers import BatchNormalization
# from kerastuner import RandomSearch

In [ ]:
vocabulary_size = 5000
one_hot_object = [one_hot(word,vocabulary_size) for word in X_data]
one_hot_object #creating one hot encoding of the object


sent_length = []
for sent in one_hot_object:
  sent_length.append(len(sent))
print(max(sent_length))  #printing maximum sentence length value to determine sentence length

36


In [ ]:
sent_length_ = 40


embedded_words = pad_sequences(
    sequences = one_hot_object,
    padding = "post",
    maxlen = sent_length_
)

embedded_words

array([[4186, 3856, 3222, ...,    0,    0,    0],
       [3272, 1379, 2602, ...,    0,    0,    0],
       [3222, 3966,  227, ...,    0,    0,    0],
       ...,
       [2488, 1614, 1531, ...,    0,    0,    0],
       [ 528, 1595, 1573, ...,    0,    0,    0],
       [2200, 3257,  232, ...,    0,    0,    0]], dtype=int32)

In [ ]:
from sklearn.model_selection import train_test_split

X_train,X_test,Y_train,Y_test = train_test_split(embedded_words,Y_data.reshape(-1,1),test_size=0.15)

In [ ]:
print(X_train.shape,X_test.shape,Y_train.shape,Y_test.shape)

(54566, 40) (9630, 40) (54566, 1) (9630, 1)


In [ ]:
embedding_vectors = 40




model = Sequential()

#embedding layer
model.add(
    Embedding(
        input_dim = vocabulary_size,
        output_dim = embedding_vectors,
        input_length = sent_length_
    )
)

#Bidirectional layer staked with LSTM
model.add(
    Bidirectional(GRU(
        units = 180,
        activation = "tanh",
        kernel_initializer = tf.keras.initializers.he_normal(),
        return_sequences = True,
        name = "Bidirectional_1_layer"
    ))
)



model.add(
    Bidirectional(GRU(
        units = 140,
        activation = "tanh",
        kernel_initializer = tf.keras.initializers.he_normal(),
        return_sequences = True,
        name = "Bidirectional_2_layer"
    ))
)



#fully connected layer 
model.add(
    Dense(
        units = 130,
        activation = "sigmoid",
        kernel_initializer = tf.keras.initializers.he_normal(),
        name = "Dense_1_layer"
    )
)


model.add(
    Dropout(rate=0.5)
)

#fully connected layer 2
model.add(
    Dense(
        units = 110,
        activation = "sigmoid",
        kernel_initializer = tf.keras.initializers.he_normal(),
        name = "Dense_2_layer"
    )
)


#batchnormalization layer
model.add(
    BatchNormalization(axis=1,center=True)
)



#fully connected layer 3
model.add(
    Dense(
        units = 42,
        activation = "sigmoid",
        kernel_initializer = tf.keras.initializers.he_normal(),
        name = "Dense_3_layer"
    )
)



model.add(
    Dropout(rate=0.5)
)


#final layer
model.add(
    Dense(
        units = 1,
        kernel_initializer = tf.keras.initializers.he_normal(),
        activation = "sigmoid"
    )
)


model.compile(
    optimizer = tf.keras.optimizers.RMSprop(learning_rate=0.0001),
    loss = "binary_crossentropy",
    metrics = ["accuracy"]
)
    

In [ ]:
model.fit(
    X_train,
    Y_train,
    epochs=25,
    batch_size=16,
    validation_split = 0.15,
    verbose = 1
)

Epoch 1/25
2899/2899 [==============================] - 53s 16ms/step - loss: 0.6972 - accuracy: 0.5432 - val_loss: 0.5636 - val_accuracy: 0.7104
Epoch 2/25
2899/2899 [==============================] - 46s 16ms/step - loss: 0.5629 - accuracy: 0.7208 - val_loss: 0.5479 - val_accuracy: 0.7261
Epoch 3/25
2899/2899 [==============================] - 46s 16ms/step - loss: 0.5400 - accuracy: 0.7366 - val_loss: 0.5487 - val_accuracy: 0.7250
Epoch 4/25
2899/2899 [==============================] - 46s 16ms/step - loss: 0.5257 - accuracy: 0.7485 - val_loss: 0.5409 - val_accuracy: 0.7303
Epoch 5/25
2899/2899 [==============================] - 46s 16ms/step - loss: 0.5230 - accuracy: 0.7478 - val_loss: 0.5434 - val_accuracy: 0.7257
Epoch 6/25
2899/2899 [==============================] - 46s 16ms/step - loss: 0.5179 - accuracy: 0.7504 - val_loss: 0.5330 - val_accuracy: 0.7325
Epoch 7/25
2899/2899 [==============================] - 46s 16ms/step - loss: 0.5129 - accuracy: 0.7549 - val_loss: 0.5337 -

In [ ]:
model.evaluate(
    X_test,
    Y_test,
    verbose=1
)

301/301 [==============================] - 2s 7ms/step - loss: 0.5773 - accuracy: 0.7255


[0.577309787273407, 0.7254883050918579]

In [ ]:
model.save("Sentiment_keras.h5")